In [ ]:
import json
import os
import shutil
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# === Step 1: Load JSON ===
with open("mock_fixer_response.json", "r") as f:
    data = json.load(f)

rates = [(k, float(v), data["base"], data["date"]) for k, v in data["rates"].items()]

# === Step 2: Schema and Spark ===
schema = StructType([
    StructField("currency", StringType(), True),
    StructField("rate", DoubleType(), True),
    StructField("base", StringType(), True),
    StructField("date", StringType(), True)
])

spark = SparkSession.builder \
    .appName("FixerMockLoad") \
    .master("local[*]") \
    .getOrCreate()

df = spark.createDataFrame(rates, schema=schema)
df.show()

# === Step 3: Save with pandas ===
pandas_df = df.toPandas()
output_file = Path("C:/temp/fixer_spark_output/fixer_rates.csv")
output_file.parent.mkdir(parents=True, exist_ok=True)
pandas_df.to_csv(output_file, index=False)

print(f"✅ Successfully saved using pandas to {output_file}")

spark.stop()


trash code by sheet p T